This notebook features some variants of models that can be used to solve Settle up problem (Dluznicek).

In [396]:
import gurobipy as g

Lets define some input parameters...

In [397]:
P = set(['A', 'B', 'N', 'J'])
c = {'A': 0, 'B': 590, 'N': 110, 'J': 300}
s = sum(c.values())/4

Let us first demonstrate the "basic" variant - the one we did in labs

In [398]:
model = g.Model()

In [399]:
f = {}
k = {}
for i in P:
    for j in P:
        f[i, j] = model.addVar(vtype=g.GRB.BINARY, obj=1)
        k[i, j] = model.addVar(vtype=g.GRB.INTEGER, lb=0)

In [400]:
for p in P:
    model.addConstr(g.quicksum(k[p, q] for q in P) + c[p] - g.quicksum(k[q, p] for q in P) == s)

Now, we want to ensure constraint if k[i, j] != 0 then f[i, j] =1

In [401]:
M = sum(c.values())
for i in P:
    for j in P:
        model.addConstr(M*f[i, j] >= k[i, j])

Next, we add the other implication: if f[i, j] = 1 then k[i, j] != 0. Note that this constraint is redundant, considering the sense of optimization. Try to take it out by yourself and see if the result changes...

In [402]:
for i in P:
    for j in P:
        model.addConstr(k[i, j] >= f[i, j])

The above is correct only if we allow trasactions to take only integer values. With "continuous" case we will deal below. 

In [403]:
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 36 rows, 32 columns and 88 nonzeros
Model fingerprint: 0x3a664b36
Variable types: 0 continuous, 32 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolved: 28 rows, 24 columns, 72 nonzeros
Variable types: 0 continuous, 24 integer (12 binary)

Root relaxation: objective 3.900000e-01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.39000    0    3    3.00000    0.39000  87.0%     -    0s
     0     0     cutoff    0         3.00000    3.00000  0.00%     -    0s

Cutting planes:
  Gomory: 1
  MIR: 4
  Flow cover: 9
  Flow path: 1
 

This is the solution and let us see the individual transactions...

In [404]:
for i in P:
    for j in P:
        if f[i, j].x > 0.5:
            print('{} -> {} sends {}'.format(i, j, k[i, j].x))

N -> B sends 140.0
A -> J sends 50.0
A -> B sends 200.0


**Model with continuous transaction values**

In [1]:
P = set(['A', 'B', 'N'])
c = {'A': 0, 'B': 590, 'N': 110}
s = sum(c.values())/3

See, now the settement value is not an integer, hence it might be necessary to consider non-integer transaction values, and maybe even transactions below 1! However, note that we do not have to consider arbitrarly precise settlement values:

In [2]:
s

233.33333333333334

How would one pay **precisely** such amount? Hence, in practice we need to restrict ourselves on the finite precision, e.g., cents:

In [3]:
s = round(s*100)/100
s

233.33

Also, this is not enough as well. Consider A paying 233.33 to B and N paying 123.33 to B. Then, B payed in total 233.34... (and not 233.33). Hence, this introduces unforseen problems. Hence, we will require that each party atendee spends *approximately* the requested amount (up to given precision)

In [430]:
model = g.Model()

In [431]:
f = {}
k = {}
for i in P:
    for j in P:
        f[i, j] = model.addVar(vtype=g.GRB.BINARY, obj=1)
        k[i, j] = model.addVar(vtype=g.GRB.CONTINUOUS, lb=0)   # now the type of "flow" variable k is a non-negative real

The ballance constraints are now given as follows (the payed amount witin s +- 1/100):

In [432]:
for p in P:
    model.addConstr(g.quicksum(k[p, q] for q in P) + 
                    round(c[p]*100)/100 - g.quicksum(k[q, p] for q in P) <= s + 1/100)
    model.addConstr(g.quicksum(k[p, q] for q in P) + 
                    round(c[p]*100)/100 - g.quicksum(k[q, p] for q in P) >= s - 1/100)

As well as the first implication:

In [433]:
M = sum(c.values())
for i in P:
    for j in P:
        model.addConstr(M*f[i, j] >= k[i, j])

Next, we add the other implication: if f[i, j] = 1 then k[i, j] != 0. It still holds that this constraint is redundant. However, as an exercise we will formulate it. In the continuous case, it is important to reason about what might be the smallest transaction...

After a bit of reasoning, it emerges that it suffice to consider transactions greater or equal to:

In [434]:
M_small = 1/100
M_small

0.01

The other constraint is (if f[i,j] is 1, then k[i,j] != 0 which in the considered precision is equivalent to k[i,j]>=M_small):

In [435]:
for i in P:
    for j in P:
        model.addConstr(k[i, j] >= M_small*f[i, j])

In [414]:
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 24 rows, 18 columns and 60 nonzeros
Model fingerprint: 0xeebba81e
Variable types: 9 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e-02, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 18 rows, 12 columns, 52 nonzeros
Variable types: 6 continuous, 6 integer (6 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: cutoff, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         2.00000    2.00000  0.00%     -    0s

Explored 0 nodes (10 simplex iterations) in 0.01 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1

And the transactions are:

In [415]:
for i in P:
    for j in P:
        if f[i, j].x > 0.5:
            print('{} -> {} sends {}'.format(i, j, k[i, j].x))

N -> B sends 123.34000000000026
A -> B sends 233.3199999999997


The numbers are meaningful up to the precision value, hence:

In [416]:
for i in P:
    for j in P:
        if f[i, j].x > 0.5:
            print('{} -> {} posle {}'.format(i, j, round(k[i, j].x*100)/100))

N -> B posle 123.34
A -> B posle 233.32


Huh, that was quite heavy for such a small change in the assignment :) This demonstrates how extremely important is to have really precise problem statement including domains of variables and parameters. 

**Alternative model, where we use negative flow values to denote who sends to who (now with just integer values :)).**

In [417]:
P = ['A', 'B', 'N', 'J']  # now the persons will have an ordering
c = [0, 590, 110, 300]
s = sum(c)/len(c)

In [418]:
model = g.Model()

In [419]:
fp = {}
fn = {}
fz = {}
k = {}
for i in range(len(c)):
    for j in range(i, len(c)):
        fp[i, j] = model.addVar(vtype=g.GRB.BINARY, obj=1, ub=1 if i != j else 0)
        fn[i, j] = model.addVar(vtype=g.GRB.BINARY, obj=1, ub=1 if i != j else 0)
        fz[i, j] = model.addVar(vtype=g.GRB.BINARY, lb=0 if i != j else 1)
        k[i, j] = model.addVar(vtype=g.GRB.INTEGER, lb=-float('inf'), name='k_{}{}'.format(P[i], P[j]))  # important! by default, Gurobi sets lower bounds to 0!

The value of flow is either positive (or zero) or negative:

In [420]:
M = sum(c)
for i in range(len(c)):
    for j in range(i, len(c)):
        model.addConstr(fp[i, j] + fn[i, j] + fz[i, j] == 1)
        model.addConstr(M*(1-fp[i, j]) + k[i, j] >= 1)
        model.addConstr(-M*(1-fn[i, j]) + k[i, j] <= -1)
        model.addConstr(M*(1-fz[i, j]) + k[i, j] >= 0)
        model.addConstr(-M*(1-fz[i, j]) + k[i, j] <= 0)

In [421]:
for i in range(len(c)):
    model.addConstr(g.quicksum(k[i, j] if i < j else -k[j, i] for j in range(len(c))) == s - c[i], name='r{}'.format(P[i]))

In [422]:
for i in range(len(c)):
    for j in range(i, len(c)):
        model.addConstr(M*fp[i, j] >= k[i, j])
        model.addConstr(M*fn[i, j] >= -k[i, j])

In [423]:
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 74 rows, 40 columns and 166 nonzeros
Model fingerprint: 0x8babaad1
Variable types: 0 continuous, 40 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 3.0000000
Presolve removed 28 rows and 16 columns
Presolve time: 0.00s
Presolved: 46 rows, 24 columns, 102 nonzeros
Variable types: 0 continuous, 24 integer (18 binary)

Root relaxation: objective 3.903904e-01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.39039    0    6    3.00000    0.39039  87.0%     -    0s

Explored 1 nodes (19 simplex iterations) in 0.01 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 3 

Optimal sol

In [425]:
for i in range(len(c)):
    for j in range(i, len(c)):
        if fp[i, j].x > 0.5 or fn[i, j].x > 0.5:
            print('{} -> {} sends {}'.format(P[i], P[j], k[i, j].x))

A -> B sends 250.0
B -> J sends -90.0
N -> J sends 140.0


Do you think that you can do the model with less binary variables? Try it as an exercise.